In [11]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility')
sys.path.append('/home/zhw272/programming/pipeline_utility/src')
from sql_setup import session
from model.center_of_mass import CenterOfMass
from toolbox.IOs.LoadComPickle import LoadComPickle
from toolbox.IOs.LoadComDatabase import LoadComDatabase
from toolbox.IOs.TransformCom import TransformCom
import numpy as np
from lib.comparison_tools import compare_lists
getcomp = LoadComPickle()
gettc = TransformCom(getcomp)
getcomd = LoadComDatabase()

In [2]:
import notebooks.Will.experimental.old.get_coms_from_pickle as getcomp
import notebooks.Will.experimental.old.get_transformed_coms as gettc

In [6]:
def query_brain_coms(brain, person_id=28, input_type_id=4,factor = np.array([0.325,0.325,20])):
    # default: person is bili, input_type is aligned
    rows = session.query(CenterOfMass)\
        .filter(CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == brain)\
        .filter(CenterOfMass.person_id == person_id)\
        .filter(CenterOfMass.input_type_id == input_type_id)\
        .all()
    row_dict = {}
    for row in rows:
        structure = row.structure.abbreviation
        row_dict[structure] = np.array([row.x, row.y, row.section])*factor
    return row_dict

In [7]:

def check_dict(d1:dict,d2:dict,verbose = True):
    if (d1.keys()==d2.keys()):
        bools = []
        for keyi in d1.keys():
            equal = np.all(np.isclose(d1[keyi],d2[keyi],atol = 1))
            if not equal and verbose:
                # print('nonequal entry:')
                print(keyi,d1[keyi],d2[keyi])
            bools.append(equal)
            
    else:
        print('unequal keys')
        compare_lists(list(d1.keys()),list(d2.keys()))
        bools = None
    return np.all(bools)

In [8]:
#compare atlas
a1 = getcomp.get_atlas_com()
a2 = getcomd.get_atlas_com()
check_dict(a1,a2) 

True

In [11]:
#compare com save vs database corrected beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_corrected_prepi_com(prepi)
    d2 = getcomd.get_corrected_prepi_com(prepi)
    print(prepi,check_dict(d1,d2) )

DK39 True
DK41 True
DK43 True
DK54 True
DK55 True
DK52 True


In [12]:
#compare com save vs database beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_prepi_com(prepi)
    d2 = getcomd.get_prepi_com(prepi)
    print(prepi,check_dict(d1,d2) )

unequal keys
in l2 not l1
{'IC', '12N'}
DK39 None
DK41 True
DK43 True
unequal keys
in l2 not l1
{'IC', 'SC'}
DK54 None
unequal keys
in l1 not l2
{'3N_R', 'Tz_L', 'Tz_R', 'LRt_R', '3N_L', '7N_L', 'Pn_R', '10N_R', 'LC_R', '5N_L', '6N_L', '10N_L', 'Amb_R', '7N_R', 'LRt_L', 'Amb_L', '5N_R', 'DC_R', '6N_R', 'LC_L', 'DC_L', 'Pn_L'}
DK55 None
unequal keys
in l2 not l1
{'Amb_R', 'Sp5C_R', 'VCP_L', 'Pn_R', 'VCP_R', 'VLL_L', 'Sp5C_L', 'VLL_R', 'Pn_L'}
DK52 None


In [13]:
#compare com Com table(old) vs save beth
prep_list = getcomp.get_prep_list()
for prepi in prep_list:
    d1 = getcomp.get_prepi_com(prepi)
    d2 = query_brain_coms(prepi, person_id=2, input_type_id=1)
    print(prepi,check_dict(d1,d2) )

DK39 True
DK41 True
DK43 True
DK54 True
DK55 True
DK52 True


In [14]:
#compare com Com table(old) vs save corrected beth
prep_list = getcomp.get_prep_list()
prep_list.remove('DK55')
for prepi in prep_list:
    d1 = getcomp.get_corrected_prepi_com(prepi)
    d2 = query_brain_coms(prepi, person_id=2, input_type_id=2)
    print(prepi,check_dict(d1,d2) )


DK39 True
DK41 True
nonequal entry:
6N_R [12704.575  6462.17   4840.   ] [12704.575   245.05   4840.   ]
DK43 False
DK54 True
DK52 True


In [12]:
beth_aligned = gettc.get_beth_coms_aligned_to_atlas()

reflection detected
reflection detected
reflection detected
reflection detected
reflection detected


In [13]:
#compare com Com table(old) vs save corrected beth
prep_list = getcomp.get_prep_list()
prep_list.remove('DK52')
id = 0
for prepi in prep_list:
    d1 = beth_aligned[id]
    d2 = query_brain_coms(prepi, person_id=28, input_type_id=2,factor = np.array([10,10,20]))
    print(prepi,check_dict(d1,d2,verbose = True) )
    id+=1

10N_L [5929.91051017 5098.8992364  2796.22026953] [6793.02 5220.86 2619.38]
10N_R [5960.90059205 5111.40146603 3134.83131441] [6849.63 5243.03 3238.28]
3N_L [4352.55493209 6025.47219634 2862.0352299 ] [3795.6  3755.17 2865.02]
3N_R [4396.17026673 6006.83238879 3002.52464011] [3878.3  3803.6  3118.52]
5N_L [4779.15198056 5231.97663809 2214.91144875] [4674.73 5047.04 1573.48]
5N_R [4767.35943527 5342.21807337 3723.28973533] [4646.53 5066.49 4337.5 ]
6N_L [5056.34119387 5181.61095113 2785.22227831] [5189.64 5184.43 2606.72]
6N_R [5054.58771518 5221.78189158 3168.45601992] [5183.13 5167.11 3310.74]
7N_L [5032.90847752 4562.62252114 2405.98091105] [5226.76 6257.44 1826.1 ]
7N_R [5023.63347241 4671.34976562 3708.83127324] [5202.28 6249.57 4215.46]
DC_L [5317.04419318 5398.91612112 1766.85275681] [5630.83  4607.78   784.142]
DC_R [5177.15384402 5518.6281616  4225.32701406] [5372.83 4764.39 5279.84]
LC_L [5145.21600585 5597.27019355 2488.01656415] [5295.15 4374.39 2125.82]
LC_R [5057.85672648 